## Load libraries

In [1]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    ConfigurableField, RunnableParallel, RunnablePassthrough
)
from llama_index.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from importlib import reload
import utils
from dotenv import load_dotenv

load_dotenv()

import warnings
warnings.filterwarnings('ignore')
#reload(utils)

### Connect to the Neo4j database 
Connect to the graph database using the [LangChain Neo4jGraph connector](https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html).

In [2]:
# Create a Neo4jGraph object to interact with the graph database
graph = Neo4jGraph()
# Clean the existing graph by removing all nodes and relationships
graph.query("MATCH (n) DETACH DELETE n")

[]

### Data Ingestion
Load Pope Francis's Wikipedia page using the [LangChain Wikipedia loader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html)

In [3]:
raw_documents = WikipediaLoader(query="Pope Francis").load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

### Construct a graph based on the retrieved documents

In [4]:
llm=ChatOllama(model="llama3")

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

### Inspect the generated graph with yfiles visualization.

In [5]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = utils.neo4j_driver
    # create a graph widget to display the graph
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget


showGraph(default_cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

## Unstructured data retriever
The `Neo4jVector.from_existing_graph` method adds both keyword and vector retrieval to documents. It configures keyword and vector search indexes for a hybrid retrieval approach.

In [6]:
# Create the vector index from the graph 

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_index = Neo4jVector.from_existing_graph(
    embedding_model,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

## Graph retriever
The graph retriever starts by identifying relevant entities in the input. For simplicity, we instruct the LLM to identify people, organizations, and locations.

In [20]:
# Define the retriever
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
)


# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


parser = JsonOutputParser(pydantic_schema=Entities)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You extract organization and person entities from the text. "
            "Return a JSON object in the following format:\n"
            "{{\n  \"names\": [\"<entity1>\", \"<entity2>\", ...]\n}}\n"
            "Only include the list of names. Do not add explanations.",
        ),
        (
            "human",
            "Text: {question}",
        ),
    ]
)
# Extract entities from text
entity_chain = prompt | llm | parser

### Test Graph retriever to detect entities in the question

In [23]:
entity_chain.invoke({"question": "Where was Pope Francis born?"}).get("names")

['Pope Francis']

### Define query generation function
Now that we can detect entities in the question, let's use a full-text index to map them to the knowledge graph. 
First, we define a full-text index and a function that will generate full-text queries that allow a bit of misspelling.

In [24]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.get("names"):
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

The `structured_retriever` function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes. 

In [25]:
print(structured_retriever("Who is Pope Francis?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Pope Francis - HEAD_OF -> Catholic Church
Pope Francis - SOVEREIGN_OF -> Vatican City State
Pope Francis - ORDAINED_AS -> Catholic Priest
Pope Francis - WORKED_IN -> Vatican City
Pope Francis - SUCCEEDED_BY -> Leo XIV
Pope Francis - SUCCEEDED_BY -> Robert Francis Prevost
Pope Francis - DIED -> 21 April 2025, Easter Monday
Pope Francis - ANNOUNCED_BY -> Cardinal Kevin Farrell
Pope Francis - ANNOUNCED_BY -> Vatican Media
Pope Francis - BURIED_AT -> Santa Maria Maggiore
Pope Francis - WORKED_AS -> Archbishop of Buenos Aires
Pope Francis - HONORED_BY -> Faustin-Archange Touadéra
Pope Francis - HONORED_BY -> Mahamat Déby
Pope Francis - HONORED_BY -> Denis Sassou Nguesso
Pope Francis - HONORED_BY -> Ismaïl Omar Guelleh
Pope Francis - HONORED_BY -> Abdel Fattah el-Sisi
Pope Francis - HONORED_BY -> Félix Tshisekedi
Pope Francis - COMMITTED_TO -> worldwide abolition of capital punishment
Pope Francis - CRITICIZED -> right-wing populism and anti-immigration politics
Pope Francis - SUPPORTED -> d

### Final retriever
Bringing it all together, combine the unstructured and graph retriever to create the final context that will be passed to an LLM.

In [26]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

### Defining the RAG chain
We have successfully implemented the the RAG retrieval component. First, the query rewriting part that allows conversational follow up questions with chat history.

In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    ),
    RunnableLambda(lambda x: x["question"]),
)

### Defining the prompt
Next, we introduce a prompt that leverages the context provided by the integrated hybrid retriever to produce the response, completing the implementation of the RAG chain.

In [28]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise. If you don't know the answer, you can say "I don't know." 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

### Test the hybrid RAG implementation

In [29]:
chain.invoke({"question": "Which is the nationality of Pope Francis?"})

Search query: Which is the nationality of Pope Francis?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Pope Francis is Argentine.'

###  Test a follow up question based on chat history - did it rewrite?
Given that we use vector and keyword search methods, we must rewrite follow-up questions to optimize our search process.

In [ ]:
chain.invoke(
    {
        "question": "When was he born?",
        "chat_history": [("Which is the nationality of Pope Francis?", "Pope Francis is Argentine.")],
    }
)

Search query: When was Pope Francis born?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Pope Francis was born Jorge Mario Bergoglio on 17 December 1936 in Flores, Buenos Aires.'

In [31]:
chain.invoke(
    {
        "question": "When did he die?",
        "chat_history": [("Which is the nationality of Pope Francis?", "Pope Francis is Argentine."),
                         ("When was he born?", "Pope Francis was born Jorge Mario Bergoglio on 17 December 1936 in Flores, Buenos Aires.")],
    }
)

Search query: When did Pope Francis die?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'He died on April 21, 2025.'